In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]='0'
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
#import seaborn as sns
#import warnings
#import csv
import gc
from time import time
import logging
#from scipy.io import loadmat
#from select_win import select_win,select_windows
import load_data as ldfile
#import create_model as cmodel
import glob
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

tf.device('/device:GPU:0')

#from scipy.signal import butter, sosfilt, sosfilt_zi, sosfiltfilt, lfilter, lfilter_zi, filtfilt, sosfreqz, resample
#from utils import hamilton_detector, christov_detector, findpeaks, engzee_detector
from ecg_detectors.ecgdetectors import Detectors, MWA, panPeakDetect, searchBack

np.random.seed(354)
#sns.set()
#warnings.filterwarnings('ignore')
#logging.getLogger("tensorflow").setLevel(logging.ERROR)
tf.random.set_seed(1342) # Set seed for reproducibility

if tf.test.gpu_device_name(): 
    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))
else:
   print("Please install GPU version of TF")


Default GPU Device:/device:GPU:0


In [2]:
with tf.device("/GPU:0"):
    t1 = time()
    input_directory = '/dataset/PhysioNet2020/Training_WFDB' # Dataset on beastie 
    train_data,train_labels_bin,val_data,val_labels_bin = ldfile.load_data2D_bin(input_directory) 
    #train_dataset,val_dataset = ldfile.load_tf(train_data,train_labels_bin,val_data,val_labels_bin)
    t2 = time()
    print('Loading data in {} seconds.'.format(round(t2-t1)))


total number of files:  6877
Number of files for trainig:  5501
Number of files for validation:  687
Number of files for testing:  687


/home/siddiqmk/.local/lib/python3.8/site-packages/scipy/signal/_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]


500 files were loaded for training!
1000 files were loaded for training!
1500 files were loaded for training!
2000 files were loaded for training!
2500 files were loaded for training!
3000 files were loaded for training!
3500 files were loaded for training!
4000 files were loaded for training!
4500 files were loaded for training!
5000 files were loaded for training!
5500 files were loaded for training!
499 files were loaded for validation!
Labels:
0. 164884008
1. 164889003
2. 164909002
3. 164931005
4. 270492004
5. 284470004
6. 426783006
7. 429622005
8. 59118001

number of test signals =  689

train_data type: <class 'list'>
train_labels  type: <class 'list'>
val_data  type: <class 'list'>
val_labels  type: <class 'list'>

train_data length 26828
train_labels length 26828
val_data length: 3532
val_labels length: 3532

train_data type: <class 'numpy.ndarray'>
train_labels_bin  type: <class 'numpy.ndarray'>
val_data  type: <class 'numpy.ndarray'>
val_labels_bin  type: <class 'numpy.ndarra

In [3]:
train_data.shape

(26828, 12, 2500, 1)

In [4]:
def create_plot(data1, i, dir_path):
    
    #plt.figure()
    #figure(num=None, figsize=(100,10), dpi = 100)
    
    #fig,lead=plt.subplots(12,1, figsize=(20,60)) #### ORIGINAL - FOR ALL 12 LEADS
    fig,lead=plt.subplots(4,1, figsize=(10,20), dpi = 20)
    #plt.gca().set_axis_off()
    
    plt.subplots_adjust(top = 1, bottom = 0, right = 1, left = 0, hspace = 0, wspace = 0) #deals with all the whitespace around the plots
    #plt.margins(0,0)

    lead[0].plot(data1[0],'black')
    lead[0].set_axis_off()
    #lead[1].plot(data1[1],'black')
    #lead[1].set_axis_off()
    #lead[2].plot(data1[2],'black')
    #lead[2].set_axis_off()
    #lead[3].plot(data1[3],'black')
    #lead[3].set_axis_off()
    lead[1].plot(data1[4],'black')
    lead[1].set_axis_off()
    #lead[5].plot(data1[5],'black')
    #lead[5].set_axis_off()
    lead[2].plot(data1[6],'black')
    lead[2].set_axis_off()
    #lead[7].plot(data1[7],'black')
    #lead[7].set_axis_off()
    #lead[8].plot(data1[8],'black')
    #lead[8].set_axis_off()
    #lead[9].plot(data1[9],'black')
    #lead[9].set_axis_off()
    #lead[10].plot(data1[10],'black')
    #lead[10].set_axis_off()
    lead[3].plot(data1[11],'black')
    lead[3].set_axis_off()
    
    file_name = 'img%d.png' %(i) 
    #fig.tight_layout()
    #plt.show()
    fig.savefig(os.path.join(dir_path,file_name), transparent = None, bbox_inches = 'tight', pad_inches = 0)
    plt.close()

STEPS:

1. Create an output folder (test to see if one exists)
2. Loop through 'train_data' -> plot images (do only 5000 at a time)

3. Save image into output folder
4. Take plotted image, open it and read it with imread (as an array item) 
5. append it to an array 'train_im'

6. Read through 'train_labels_bin' and copy section (5000 at a time) into 'train_lab'

7. Repeat process for 'val_data' and 'val_labels_bin'

8. Make train_dataset and val_dataset from data and labels combined.

### TRAINING DATA

In [5]:
# test block to test the function in step 1:
dir_path = "train_outputs_v1"
os.makedirs(dir_path, exist_ok=True)

In [6]:
# test block to test the loop in step 2 & 3:
with tf.device("/GPU:0"):
    t1 = time()
    for i,j in zip (range(0,26828), range(100000,126828)):  #the second range is for the file numbers you're gonna generate in the 
        test_1 = create_plot(train_data[i], j, dir_path)

    t2 = time()
    print('Loading data in {} seconds.'.format(round(t2-t1)))

Loading data in 1678 seconds.


In [7]:
# test blcok to test saving the img in step 4 & 5:
with tf.device("/GPU:0"):
    t1 = time()
    X_data = []
    files = sorted(glob.glob('train_outputs_v1/*.png')) #change temp_outputs to your temporary storage area for images created
    for myFile in files:
        print(myFile)
        image = cv2.imread (myFile)
        X_data.append (image)
    train_im = np.asarray(X_data)


    t2 = time()
    print('Loading data in {} seconds.'.format(round(t2-t1)))

train_outputs_v1/img100000.png
train_outputs_v1/img100001.png
train_outputs_v1/img100002.png
train_outputs_v1/img100003.png
train_outputs_v1/img100004.png
train_outputs_v1/img100005.png
train_outputs_v1/img100006.png
train_outputs_v1/img100007.png
train_outputs_v1/img100008.png
train_outputs_v1/img100009.png
train_outputs_v1/img100010.png
train_outputs_v1/img100011.png
train_outputs_v1/img100012.png
train_outputs_v1/img100013.png
train_outputs_v1/img100014.png
train_outputs_v1/img100015.png
train_outputs_v1/img100016.png
train_outputs_v1/img100017.png
train_outputs_v1/img100018.png
train_outputs_v1/img100019.png
train_outputs_v1/img100020.png
train_outputs_v1/img100021.png
train_outputs_v1/img100022.png
train_outputs_v1/img100023.png
train_outputs_v1/img100024.png
train_outputs_v1/img100025.png
train_outputs_v1/img100026.png
train_outputs_v1/img100027.png
train_outputs_v1/img100028.png
train_outputs_v1/img100029.png
train_outputs_v1/img100030.png
train_outputs_v1/img100031.png
train_ou

In [8]:
print(type(train_im))
print(train_im.shape)

<class 'numpy.ndarray'>
(26828, 400, 200, 3)


In [9]:
np.save('train_v1.npy', train_im)
del train_im, X_data
gc.collect()

63652

In [10]:
train_labels_bin.shape

(26828, 9)

In [11]:
# test block for step 6:

start = 0
end = 26828

rows, cols = (26828, 9) 
temp_arr = [[0]*cols]*rows 

for i, j in zip (range(0,26828), range(start, end)):    
    temp_arr[i][:] = train_labels_bin[j][:];    
    print(i, j)

train_lab = np.asarray(temp_arr)
train_lab = train_lab.astype('uint8') 

np.save('train_lab_v1.npy', train_lab)
del train_lab
gc.collect()

0 0
1 1
2 2
3 3
4 4
5 5
6 6
7 7
8 8
9 9
10 10
11 11
12 12
13 13
14 14
15 15
16 16
17 17
18 18
19 19
20 20
21 21
22 22
23 23
24 24
25 25
26 26
27 27
28 28
29 29
30 30
31 31
32 32
33 33
34 34
35 35
36 36
37 37
38 38
39 39
40 40
41 41
42 42
43 43
44 44
45 45
46 46
47 47
48 48
49 49
50 50
51 51
52 52
53 53
54 54
55 55
56 56
57 57
58 58
59 59
60 60
61 61
62 62
63 63
64 64
65 65
66 66
67 67
68 68
69 69
70 70
71 71
72 72
73 73
74 74
75 75
76 76
77 77
78 78
79 79
80 80
81 81
82 82
83 83
84 84
85 85
86 86
87 87
88 88
89 89
90 90
91 91
92 92
93 93
94 94
95 95
96 96
97 97
98 98
99 99
100 100
101 101
102 102
103 103
104 104
105 105
106 106
107 107
108 108
109 109
110 110
111 111
112 112
113 113
114 114
115 115
116 116
117 117
118 118
119 119
120 120
121 121
122 122
123 123
124 124
125 125
126 126
127 127
128 128
129 129
130 130
131 131
132 132
133 133
134 134
135 135
136 136
137 137
138 138
139 139
140 140
141 141
142 142
143 143
144 144
145 145
146 146
147 147
148 148
149 149
150 150
151 151
152 

0

array([[0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

### VALIDATION DATA

In [12]:
# test block to test the function in step 1:
dir_path = "val_outputs_v1"
os.makedirs(dir_path, exist_ok=True)

In [13]:
# test block to test the loop in step 2 & 3:
t1 = time()
for i,j in zip (range(0,3532), range(10000,13532)):
    test_1 = create_plot(val_data[i], j, dir_path)
    
t2 = time()
print('Loading data in {} seconds.'.format(round(t2-t1)))

Loading data in 221 seconds.


In [14]:
# test blcok to test saving the img in step 4 & 5:

t1 = time()
X_data = []
files = sorted(glob.glob('val_outputs_v1/*.png')) #change temp_outputs to your temporary storage area for images created
for myFile in files:
    print(myFile)
    image = cv2.imread (myFile)
    X_data.append (image)
val_im = np.asarray(X_data)

t2 = time()
print('Loading data in {} seconds.'.format(round(t2-t1)))

val_outputs_v1/img10000.png
val_outputs_v1/img10001.png
val_outputs_v1/img10002.png
val_outputs_v1/img10003.png
val_outputs_v1/img10004.png
val_outputs_v1/img10005.png
val_outputs_v1/img10006.png
val_outputs_v1/img10007.png
val_outputs_v1/img10008.png
val_outputs_v1/img10009.png
val_outputs_v1/img10010.png
val_outputs_v1/img10011.png
val_outputs_v1/img10012.png
val_outputs_v1/img10013.png
val_outputs_v1/img10014.png
val_outputs_v1/img10015.png
val_outputs_v1/img10016.png
val_outputs_v1/img10017.png
val_outputs_v1/img10018.png
val_outputs_v1/img10019.png
val_outputs_v1/img10020.png
val_outputs_v1/img10021.png
val_outputs_v1/img10022.png
val_outputs_v1/img10023.png
val_outputs_v1/img10024.png
val_outputs_v1/img10025.png
val_outputs_v1/img10026.png
val_outputs_v1/img10027.png
val_outputs_v1/img10028.png
val_outputs_v1/img10029.png
val_outputs_v1/img10030.png
val_outputs_v1/img10031.png
val_outputs_v1/img10032.png
val_outputs_v1/img10033.png
val_outputs_v1/img10034.png
val_outputs_v1/img10

In [15]:
type(val_im)
val_im.shape

(3532, 400, 200, 3)

In [16]:
np.save('val_v1.npy', val_im)
del X_data, val_im
gc.collect()
    

14197

In [17]:
val_labels_bin.shape

(3532, 9)

In [18]:
# test block for step 6:

start = 0
end = 3532

rows, cols = (3532, 9) 
temp_arr = [[0]*cols]*rows 

for i, j in zip (range(0,3532), range(start, end)):    
    temp_arr[i][:] = val_labels_bin[j][:];    
    print(i, j)

val_lab = np.asarray(temp_arr)
val_lab = val_lab.astype('uint8') 

np.save('val_lab_v1.npy', val_lab)
del val_lab
gc.collect()

0 0
1 1
2 2
3 3
4 4
5 5
6 6
7 7
8 8
9 9
10 10
11 11
12 12
13 13
14 14
15 15
16 16
17 17
18 18
19 19
20 20
21 21
22 22
23 23
24 24
25 25
26 26
27 27
28 28
29 29
30 30
31 31
32 32
33 33
34 34
35 35
36 36
37 37
38 38
39 39
40 40
41 41
42 42
43 43
44 44
45 45
46 46
47 47
48 48
49 49
50 50
51 51
52 52
53 53
54 54
55 55
56 56
57 57
58 58
59 59
60 60
61 61
62 62
63 63
64 64
65 65
66 66
67 67
68 68
69 69
70 70
71 71
72 72
73 73
74 74
75 75
76 76
77 77
78 78
79 79
80 80
81 81
82 82
83 83
84 84
85 85
86 86
87 87
88 88
89 89
90 90
91 91
92 92
93 93
94 94
95 95
96 96
97 97
98 98
99 99
100 100
101 101
102 102
103 103
104 104
105 105
106 106
107 107
108 108
109 109
110 110
111 111
112 112
113 113
114 114
115 115
116 116
117 117
118 118
119 119
120 120
121 121
122 122
123 123
124 124
125 125
126 126
127 127
128 128
129 129
130 130
131 131
132 132
133 133
134 134
135 135
136 136
137 137
138 138
139 139
140 140
141 141
142 142
143 143
144 144
145 145
146 146
147 147
148 148
149 149
150 150
151 151
152 

0

In [15]:
val_lab.shape

dtype('uint8')

In [22]:
import PIL
image = PIL.Image.open("train_outputs_v1/img10000.png")
width, height = image.size
print(width, height)

100 200
